In [11]:
!pip install pandas numpy matplotlib seaborn plotly scikit-learn

In [3]:
import pandas as pd
import os
import datetime

In [5]:

# Define filename
filename = "budget_data.csv"

# Read the CSV file
try:
    df = pd.read_csv(filename)
    print("\n📂 Budget Data Loaded Successfully!\n")
    print(df.head())  # Display first few rows
except FileNotFoundError:
    print(f"\n❌ Error: The file '{filename}' does not exist. Please enter data first!")
except pd.errors.EmptyDataError:
    print(f"\n⚠️ Warning: The file '{filename}' is empty!")



📂 Budget Data Loaded Successfully!

  Month_Year     Category   Spent    Limit  Over Budget
0      01/24         Rent  6000.0  10000.0        False
1      01/24  Electricity  4000.0   5000.0        False
2      01/24        Water  1500.0   2000.0        False
3      01/24         Food  6000.0   8000.0        False
4      01/24     Clothes   7000.0   8000.0        False


In [13]:

# Function to read last month's data
def get_last_month_data(filename="budget_data.csv"):
    if os.path.exists(filename):
        df = pd.read_csv(filename)
        if not df.empty:
            return df.iloc[-1]  # Get the last row (last month’s data)
    return None  # If file doesn't exist or is empty

# Function to get user input while fetching categories from file
def get_user_input(last_data):
    month_year = input("Enter the budget month (MM/YY): ").strip()

    # Validate Date Format
    try:
        datetime.datetime.strptime(month_year, "%m/%y")
    except ValueError:
        print("❌ Invalid format! Please enter in MM/YY format.")
        return get_user_input(last_data)
    
    total_money = float(input("Enter your total money: "))

    # Fetch previous savings and allow additional savings
    prev_savings = last_data["Updated Savings"] if last_data is not None else 0
    print(f"\n💰 Previous savings: {prev_savings}")
    additional_savings = float(input("Enter any additional savings for this month: "))
    total_savings = prev_savings + additional_savings

    min_savings_limit = float(input("Enter the minimum savings limit: "))

    # Fetch categories from last month's data
    categories = {}
    if last_data is not None:
        print("\n📌 Fetching previous expense categories...")
        for column in last_data.index:
            if column not in ["Username", "Date", "Total Money", "Total Expenses", "Remaining Money", 
                              "Updated Savings", "Min Savings Limit", "Savings Goal", "Goal Cost", "Amount Needed for Goal"]:
                spent = float(input(f"Enter expenditure for {column} (Last: {last_data[column]}): "))
                categories[column] = {"spent": spent, "limit": last_data[column]}  # Keep the old limit

    # Allow updating the savings goal
    goal_item = input("Enter a new savings goal (or press Enter to keep the last one): ").strip()
    if not goal_item and last_data is not None:
        goal_item = last_data["Savings Goal"]
        goal_cost = last_data["Goal Cost"]
    else:
        goal_cost = float(input(f"Enter the cost of {goal_item}: "))

    savings_goal = {"Item": goal_item, "Cost": goal_cost}

    return month_year, total_money, total_savings, min_savings_limit, categories, savings_goal

# Function to append new month's data to CSV
def append_monthly_budget_data(filename="budget_data.csv"):
    # Retrieve last month's data
    last_data = get_last_month_data(filename)

    # Get user input
    month_year, total_money, savings, min_savings_limit, expenses, savings_goal_item = get_user_input(last_data)

    # Calculate total expenses and remaining money
    total_expenses = sum(exp["spent"] for exp in expenses.values()) if expenses else 0
    remaining_money = total_money - total_expenses
    savings += remaining_money  # Add remaining money to savings

    # Prepare data dictionary
    new_data = {
        "Username": last_data["Username"] if last_data is not None else input("Enter your name: ").strip(),
        "Date": month_year,
        "Total Money": total_money,
        "Total Expenses": total_expenses,
        "Remaining Money": remaining_money,
        "Updated Savings": savings,
        "Min Savings Limit": min_savings_limit,
        "Savings Goal": savings_goal_item["Item"],
        "Goal Cost": savings_goal_item["Cost"],
        "Amount Needed for Goal": max(0, savings_goal_item["Cost"] - savings),
    }

    # Add expense categories
    for category, values in expenses.items():
        new_data[category] = values["spent"]

    # Convert new data to DataFrame
    new_df = pd.DataFrame([new_data])

    # Read existing CSV and append new data
    if os.path.exists(filename):
        try:
            existing_df = pd.read_csv(filename)
            updated_df = pd.concat([existing_df, new_df], ignore_index=True)  # Append new data
        except pd.errors.EmptyDataError:
            updated_df = new_df  # If file exists but is empty, use new data only
    else:
        updated_df = new_df  # If file doesn't exist, create a new one

    # Save updated data back to CSV
    updated_df.to_csv(filename, index=False)
    print(f"\n✅ Data for {month_year} added successfully to '{filename}'!")

# Run the function
append_monthly_budget_data()

Enter the budget month (MM/YY):  10/23
Enter your total money:  20000


KeyError: 'Updated Savings'